# Writing to docx
This notebook shows some ways of putting the transcribed audio into docx files (word files). This is often a preferable format for the end user. The process relies on the library `python-docx`, which has functionality to read from and write to docx files.

In [ ]:
%pip install python-docx

In [6]:
import os
from docx import Document



folder_path = os.path.join("Data", "Example 2")
if not os.path.exists(folder_path):
    print("Folder does not exist")

In [7]:
# Make a docx file from the linebreak separated text file

file_name = "default_whisper_20230124_large-v2_radbryte.txt"

with open(os.path.join(folder_path, file_name), "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f.readlines()]

document = Document()

document.add_heading(f"Transcription", 0)
#document.add_page_break()

p = document.add_paragraph()

for line in lines:
    run = p.add_run(str(line)+" ")
    run.add_break()
    #font = run.font
    #font.color.rgb = shared.RGBColor(200, 0, 0)

document.save(os.path.join(folder_path, "Transcription.docx"))